In [1]:
# loading tensorflow model

In [3]:
# import libraries
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, HalvingRandomSearchCV

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, GRU, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.callbacks import EarlyStopping

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [43]:
counties_df = pd.read_csv('../clean_data/counties_models.csv')

In [44]:
counties_df.head()

,county,date,7dayrollingavg_newlyconfirmed,7dayrollingavg_newdeath,hospitalized_covid_patients,all_hospital_beds,icu_available_beds
0,Alameda,2020-03-29,1.153915,0.042738,127.0,2725.0,119.0
1,Alameda,2020-03-30,1.384698,0.051285,148.0,3283.0,146.0
2,Alameda,2020-03-31,1.478721,0.042738,150.0,1857.0,77.0
3,Alameda,2020-04-01,1.307770,0.042738,128.0,2456.0,106.0
4,Alameda,2020-04-02,1.478721,0.051285,133.0,2580.0,112.0


In [45]:
counties_df['date'] = pd.to_datetime(counties_df['date'])

In [46]:
counties_df.set_index(['county', 'date'], inplace = True)
counties_df.head()

7dayrollingavg_newlyconfirmed  7dayrollingavg_newdeath  \
county  date                                                                 
Alameda 2020-03-29                       1.153915                 0.042738   
        2020-03-30                       1.384698                 0.051285   
        2020-03-31                       1.478721                 0.042738   
        2020-04-01                       1.307770                 0.042738   
        2020-04-02                       1.478721                 0.051285   

                    hospitalized_covid_patients  all_hospital_beds  \
county  date                                                         
Alameda 2020-03-29                        127.0             2725.0   
        2020-03-30                        148.0             3283.0   
        2020-03-31                        150.0             1857.0   
        2020-04-01                        128.0             2456.0   
        2020-04-02                        133.0             2580.0   

                    icu_available_beds  
county  date                            
Alameda 2020-03-29               119.0  
        2020-03-30               146.0  
        2020-03-31                77.0  
        2020-04-01               106.0  
        2020-04-02               112.0

In [47]:
county_select = 'Los Angeles'

In [48]:
selected_df = counties_df.loc[county_select]
features = ['hospitalized_covid_patients']
X = selected_df[features]
y = selected_df[['7dayrollingavg_newlyconfirmed']].values

In [49]:
# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = False, test_size = 0.15)

In [50]:
# scale our data
mms = MinMaxScaler()
X_train_sc = mms.fit_transform(X_train)
X_test_sc = mms.transform(X_test)

In [62]:
new_model = load_model('county_models/Alameda.h5')

In [63]:
# generate predictions
future_pred_count = 1
future = []
batch = X_test_sc[-future_pred_count:].reshape((1, future_pred_count, 1))
new_model.predict(batch)[0].shape

for i in range(22):
    pred = new_model.predict(batch)[0]
    future.append(pred)
    batch = np.append(batch[:, 0:, :], [[pred[i]]] , axis = 1)

In [64]:
selected_df.index

DatetimeIndex(['2020-03-29', '2020-03-30', '2020-03-31', '2020-04-01',
               '2020-04-02', '2020-04-03', '2020-04-04', '2020-04-05',
               '2020-04-06', '2020-04-07',
               ...
               '2021-01-15', '2021-01-16', '2021-01-17', '2021-01-18',
               '2021-01-19', '2021-01-20', '2021-01-21', '2021-01-22',
               '2021-01-23', '2021-01-24'],
              dtype='datetime64[ns]', name='date', length=302, freq=None)

In [65]:
dates = [selected_df.index[-1] + pd.tseries.offsets.DateOffset(days = x) for x in range(0, 23)]
future_date = pd.DataFrame(index = dates[1:], columns = X.columns)
future_date.tail()

,hospitalized_covid_patients
2021-02-11,NaN
2021-02-12,NaN
2021-02-13,NaN
2021-02-14,NaN
2021-02-15,NaN


In [66]:
df_predict = pd.DataFrame(future[21], index = future_date[-22:].index, columns = ['Prediction'])
df_predict.head()

,Prediction
2021-01-25,17.314999
2021-01-26,21.175776
2021-01-27,21.845348
2021-01-28,22.023079
2021-01-29,22.084812
